In [1]:
import pandas as pd
import csv
import re
import string
import nltk

from pythainlp.tokenize import Tokenizer
from pythainlp import word_tokenize
from pythainlp.corpus import thai_stopwords
from pythainlp.tokenize import dict_trie
from pythainlp.tag import pos_tag

from nltk.corpus import words
from stop_words import get_stop_words

# Function

In [3]:
# word correction for EN
#parser = GingerIt()

# download stopwords of TH and EN
nltk.download('words')
th_stop = tuple(thai_stopwords())
en_stop = tuple(get_stop_words('en'))

# clean the text
def clean_text(msg): 

    msg = str(msg).lower()
    msg = re.sub(r"\,", "", str(msg))
    msg = re.sub(r"[^a-z0-9ก-๙\.\%\:]", " ", str(msg))
    
    # Delete punctuation
   # str_punc = string.punctuation.replace(".","")
   # for c in str_punc:
    #    msg = re.sub(r'\{}'.format(c),'',str(msg))
    
    # ลบ separator เช่น \n \t
    msg = ' '.join(msg.split())
    
    return msg

[nltk_data] Downloading package words to
[nltk_data]     C:\Users\super\AppData\Roaming\nltk_data...
[nltk_data]   Package words is already up-to-date!


# Load data

## Product details

In [2]:
def get_data_from_csv(file_name):
    
    FILE = './Data/{}.csv'.format(file_name)
    df_1 = pd.read_csv(FILE,index_col=0)
    return df_1

In [9]:
# Load data from csv 

file_name = 'data_jib_1'
file_name_2 = 'data_jib_2'
prod_name = 'productname'
prod_name_2 = 'mistake_get_data'

df_1 = get_data_from_csv(file_name)
df_2 = get_data_from_csv(file_name_2)
prod_name_df = get_data_from_csv(prod_name)
prod_name_df_2 = get_data_from_csv(prod_name_2)

In [15]:
df_1.head()

,P_0Q,P_0A,P_1Q,P_1A,P_2Q,P_2A,P_3Q,P_3A,P_4Q,P_4A,...,P_3882Q,P_3882A,P_3883Q,P_3883A,P_3884Q,P_3884A,P_3885Q,P_3885A,P_3886Q,P_3886A
0,CPU Brand,AMD,CPU Brand,Intel,CPU Brand,AMD,CPU Brand,Intel,CPU Brand,Intel,...,Model,INTEL 1151 CORE I5-9400F 2.90 GHz,Model,INTEL 1151 CORE I5-9600KF 3.70 GHz,Model,AMD AM4 RYZEN 5 3600 3.6GHz,Model,CPU INTEL 1151 CORE I7-9700KF,Model,AMD AM4 RYZEN 7 3700X 3.6GHz
1,CPU Model,AMD Ryzen 5 3500U,CPU Model,Intel Core i5-10210U,CPU Model,AMD Ryzen 5 3500U,CPU Model,Intel Core i5-1035G1,CPU Model,Intel Core i5-9300H,...,CPU Core / Thread,6/6,CPU Core / Thread,6/6,CPU Core / Thread,6/12,CPU Core / Thread,8/8,CPU Core / Thread,8/16
2,Base Clock,2.00 GHz,Base Clock,1.60 GHz,Base Clock,2.10 GHz,Base Clock,1.00 GHz,Base Clock,2.40 GHz,...,Integrated Graphics,-,Integrated Graphics,-,Integrated Graphics,-,Integrated Graphics,-,Integrated Graphics,-
3,Boost Clock,3.60 GHz,Boost Clock,4.20 GHz,Boost Clock,3.70 GHz,Boost Clock,3.60 GHz,Boost Clock,4.10 GHz,...,Model,GIGABYTE B365 M AORUS ELITE,Model,ID COOLING AURAFLOW X 240 RGB,Model,ASROCK B450 STEEL LEGEND,Model,DEEPCOOL CASTLE 280 ARGB,Model,GIGABYTE B450 AORUS ELITE
4,Cache,4 MB,Cache,6 MB,Cache,4 MB,Cache,6 MB,Cache,8 MB,...,Chipset,B365,Model,MSI Z390-A PRO,Chipset,B450,Model,GIGABYTE Z390 AORUS ELITE,Chipset,B450


In [11]:
df_2.head()

,P_101Q,P_101A,P_102Q,P_102A,P_103Q,P_103A,P_104Q,P_104A,P_105Q,P_105A,...,P_4060Q,P_4060A,P_4061Q,P_4061A,P_4062Q,P_4062A,P_4063Q,P_4063A,P_4064Q,P_4064A
0,Detail,Spin Speed : 2200+/-10 RPM,Detail,Dimension : 128.5 x 140 x 168.3 mm<end line>Fa...,Detail,Spin Speed : 2200 RPM<end line>Dimensions : 11...,Detail,Product Name : Wraith Ripper<end line>Product ...,Detail,Max. Rotational Speed (+/- 10%) 1500 RPM<end l...,...,Detail,รูปแบบ : WD DRPP-UPGRADE TO 3 Year<end line>อุ...,Detail,รูปแบบ : DRPP-FULL 3 Year<end line>อุปกรณ์ : L...,Detail,รูปแบบ : DRPP-LITE 1 Year<end line>อุปกรณ์ : L...,Detail,รูปแบบ : DRPP-FULL 1 Year<end line>อุปกรณ์ : L...,Detail,รูปแบบ : DRPP-LITE 3 Year<end line>อุปกรณ์ : L...
1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


## Product names

In [12]:
prod_name_df.head()

,index,Product_Name
0,0,NOTEBOOK (โน้ตบุ๊ค) ASUS VIVOBOOK 15 X512DA-EJ...
0,1,NOTEBOOK (โน้ตบุ๊ค) ACER ASPIRE 3 A315-55G-53C...
0,2,NOTEBOOK (โน้ตบุ๊ค) ASUS VIVOBOOK 15 X512DA-EJ...
0,3,NOTEBOOK (โน้ตบุ๊ค) HP 15S-FQ1012TU (SILVER)
0,4,NOTEBOOK (โน้ตบุ๊ค) DELL INSPIRON 7591-W567015...


In [13]:
prod_name_df_2.head()

,Unnamed: 0.1,Name,No.,link
0,0,CPU AIR COOLER (พัดลมซีพียู) DEEPCOOL ICEEDGE ...,101,https://www.jib.co.th/web/product/readProduct/...
1,0,CPU AIR COOLER (พัดลมซีพียู) CRYORIG R1 UNIVERSAL,102,https://www.jib.co.th/web/product/readProduct/...
2,0,CPU AIR COOLER (พัดลมซีพียู) ID COOLING SE-802,103,https://www.jib.co.th/web/product/readProduct/...
3,0,CPU AIR COOLER (พัดลมซีพียู) COOLER MASTER WRA...,104,https://www.jib.co.th/web/product/readProduct/...
4,0,CPU AIR COOLER (พัดลมซีพียู) NOCTUA NH-D15S,105,https://www.jib.co.th/web/product/readProduct/...


# Extract attributes from dataframe to triples

In [49]:
for idx in df_1:
    for attr in df_1[idx]:
        if not pd.isna(attr):
            print(attr)
        
    break

CPU Brand
CPU Model
Base Clock
Boost Clock
Cache
Graphic Chip
Brand
Display Size
Display Resolution
Display Type
Touchscreen
Ram
Ram Type
HDD 1 Capacity
HDD 1 RPM
HDD 1 Type
HDD 2 Capacity
HDD 2 RPM
HDD 2 Type
HDD 3 Capacity
HDD 3 RPM
HDD 3 Type
HDD 4 Capacity
HDD 4 RPM
HDD 4 Type
Optical Disk Drive
Wi-Fi
Bluetooth
LAN
Sim Card (Cellular Data)
OS
Web Camera
Battery
Warranty
USB 2.0
USB 3.0
USB 3.1
USB 3.2
USB-C
DisplayPort
HDMI
D-Sub
Thunderbolt
Card Reader
Finger Print Reader
Weight
Color
Additional
Latest Update


In [58]:
def extract_attr_df_1(df):
    attr_val_dict = {}
    relations = []

    regex_attr = re.compile("P\_\d+Q")
    regex_val = re.compile("P\_\d+A")
    regex_idx = re.compile("\d+")
    
    for idx in df:
        idx_prod = re.findall(regex_idx, idx)[0]
        
        if re.match(regex_attr, idx):
            tmp_list = []
            for attr in df[idx]:
                if not pd.isna(attr):
                    tmp_list.append(attr)
                    relation.append(attr)
                
        elif re.match(regex_val, idx):
            attr_val_dict[idx_prod] = []
            for val in df[idx]:
                if not pd.isna(val):
                    tmp_list.append(val)
            for i in range(len(tmp_list))

    return attr_val_dict

In [59]:
attr_val_1 = extract_attr_df_1(df_1)

In [73]:
reg = re.compile("\d+")
test = re.findall(reg,"P_101Q")